# Исследование надёжности заёмщиков

---
**Цель проекта:** разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

---

Используемые данные:

bank_data - данные о клиентах банка:

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита
    

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

## Предобработка данных

In [2]:
# bank_data = pd.read_csv('/Users/dianamini/Developer/Data/data_1.csv')
# bank_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [3]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

- Минусовые значения в столбце "days_employed"  
- Грубые дубликаты в столбце "education"

### Обработка пропусков

In [4]:
bank_data[bank_data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [5]:
groups = bank_data.groupby(["income_type", "education_id"])
bank_data['total_income'].fillna(groups['total_income'].transform('median'),inplace=True)

In [6]:
groups = bank_data.groupby(["dob_years", "gender"])
bank_data['days_employed'].fillna(groups['days_employed'].transform('median'),inplace=True)

In [7]:
bank_data[bank_data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [8]:
bank_data[bank_data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


- Сгруппировала по двум характеристикам для каждого значения, заменила пропуски на медианы в столбцах "total_income" и "days_employed". Проверила методом isna() наличие нулей, теперь их нет. 

In [9]:
bank_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [10]:
def value_abs(value):
    return abs(value)
bank_data['days_employed'] = bank_data['days_employed'].apply(value_abs)
bank_data['children'] = bank_data['children'].apply(value_abs)
bank_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Нахожу пропущенные значения в двух столбцах "days_employed" и "total_income". 
В обоих пропущенные значения типа float64(NaN), значить мне нужно заменить его на 0, чтобы не получить ошибок в дальнейшем.
Причины появления пропусков в таблице: отсутствие стажа = отсутстсвие ежемесечного дохода; скорее всего эти строки не были заполнены при подаче заявки на кредит.

### Замена типа данных

In [11]:
bank_data['days_employed'] = bank_data['days_employed'].astype('int64')

In [12]:
bank_data['total_income'] = bank_data['total_income'].astype('int64')

In [13]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Заменяю тип данных столбцов days_employed и total_income с float64 на int64 для большей наглядности можно пожертвовать некоторой точносью, потому что все эти числа после запятой мне не пригодятся и только запутают.
Был выбран метод astype(), так как он позволяет менять численные данные на целочисленные и наоборот. Остальные методы вызывают ошибку.

### Обработка дубликатов

In [14]:
bank_data['education'] = bank_data['education'].str.lower()

In [15]:
bank_data.duplicated().sum()

71

In [16]:
bank_data.loc[bank_data.duplicated(), :]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1825,41,среднее,1,женат / замужем,0,F,сотрудник,0,136555,покупка жилья для семьи
3290,0,340298,58,среднее,1,гражданский брак,1,F,пенсионер,0,114842,сыграть свадьбу
4182,1,1631,34,высшее,0,гражданский брак,1,F,сотрудник,0,165640,свадьба
4851,0,352450,60,среднее,1,гражданский брак,1,F,пенсионер,0,114842,свадьба
5557,0,340298,58,среднее,1,гражданский брак,1,F,пенсионер,0,114842,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,358068,64,среднее,1,женат / замужем,0,F,пенсионер,0,114842,дополнительное образование
21032,0,352450,60,среднее,1,женат / замужем,0,F,пенсионер,0,114842,заняться образованием
21132,0,2314,47,среднее,1,женат / замужем,0,F,сотрудник,0,136555,ремонт жилью
21281,1,1421,30,высшее,0,женат / замужем,0,F,сотрудник,0,165640,покупка коммерческой недвижимости


In [17]:
bank_data = bank_data.drop_duplicates()

In [18]:
bank_data = bank_data.loc[bank_data['children'] < 4]

In [19]:
bank_data.duplicated().sum()

0

**Вывод**

Посмотрела общее количество дубликатов в датафрейме, решила посмотреть на строки с полными дубликатами, использовала метод duplicated() в логическом условии.

Методом drop_duplicates() удалила дублирующие строки.
При повторной проверке количества дубликатов убедилась, что все удалены.

Вижу несколько причин появления дубликатов:
- Так как во всех строках не указаны стаж и доход, значит есть вероятность того, что заявки подавали неоднократно и они не были одобрены
- Ошибка в системе банка
- Ошибка при формировании списка

Так же в столбце education привела все строки к нижнему регистру, чтобы легче было разбивать по категориям в дальнейшем

### Лемматизация

In [20]:
bank_data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [21]:
m = Mystem()

def lemmatize_text(text):
    return m.lemmatize(text)

lemmas = bank_data['purpose'].apply(lemmatize_text)

big_list = []
for sublist in lemmas:
    for item in sublist:
        if len(item) > 3:
            big_list.append(item)
    
Counter(big_list)

Counter({'покупка': 5861,
         'жилье': 4432,
         'приобретение': 461,
         'автомобиль': 4278,
         'дополнительный': 901,
         'образование': 3990,
         'сыграть': 761,
         'свадьба': 2311,
         'операция': 2588,
         'проведение': 762,
         'семья': 635,
         'недвижимость': 6317,
         'коммерческий': 1302,
         'жилой': 1224,
         'строительство': 1866,
         'собственный': 627,
         'подержать': 470,
         'свой': 2220,
         'заниматься': 899,
         'сделка': 936,
         'подержанный': 481,
         'получение': 1309,
         'высокий': 1364,
         'профильный': 434,
         'сдача': 649,
         'ремонт': 605})

In [22]:
def select_category(purpose): 
    lemma = m.lemmatize(purpose)
    if 'жилье' in lemma or 'недвижимость' in lemma:
        if 'ремонт' in lemma:
            return 'Ремонт недвижимости'
        return 'Покупка недвижимости'
    
    if 'автомобиль' in lemma:
        return 'Покупка автомобиля'
    
    if 'образование' in lemma:
        return 'Оплата обучения'
    
    if 'свадьба' in lemma:
        return 'Проведение мероприятий'
    
    return 'Не удалось определить'
    
bank_data['purpose_category'] = bank_data['purpose'].apply(select_category)

In [23]:
bank_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Покупка недвижимости
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Покупка автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Покупка недвижимости
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Оплата обучения
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Проведение мероприятий


**Вывод**

- Для начала применила метод unique(), чтобы видеть все возможные цели кредита.
- Применила метод Counter(), увидела колличество ключевых слов в столбце. Для удобства отсеяла предлоги и пробелы, чтоб не мешали.
- На основе полученных ключевых слов составила функцию select_category с условиями присвоения категорий. Категории целей кредита выделила в отдельном столбце purpose_category.

### Категоризация данных

In [24]:
pivot_children_debt = bank_data.pivot_table(values = ['debt'], aggfunc = 'mean', index = ['children'], fill_value=0)
pivot_children_debt

,debt
children,
0,0.075438
1,0.091658
2,0.094542
3,0.081818


Приняла решение удалить из списка всех у кого 20 детей. Так как их всего 76 штук, то это не такая страшная потеря и она ни на что не влияет. Так же удалила строки людей с 5ю детьми, так как их было всего 41 и процент задолженности был нулевой, он очень мешал анализу.

In [25]:
pivot_purpose_debt = bank_data.pivot_table(values = ['debt'], aggfunc = 'mean', index = ['purpose_category'], fill_value=0)
pivot_purpose_debt

,debt
purpose_category,
Оплата обучения,0.092481
Покупка автомобиля,0.093502
Покупка недвижимости,0.073147
Проведение мероприятий,0.079186
Ремонт недвижимости,0.057851


In [26]:
family_status_df = bank_data.drop_duplicates(["family_status_id"])
family_status_dict = family_status_df.set_index(['family_status_id'])['family_status'].to_dict()

In [27]:
pivot_family_status_debt = bank_data.pivot_table(values = ['debt'], aggfunc = 'mean', index = ['family_status'], fill_value=0)
pivot_family_status_debt

,debt
family_status,
Не женат / не замужем,0.097178
в разводе,0.070470
вдовец / вдова,0.066038
гражданский брак,0.093243
женат / замужем,0.075486


In [28]:
def select_income_category(income):
    if income == 0:
        return 'Нулевой'
    if income <= 50000:
        return 'Низкий'
    if income <= 150000:
        return 'Средний'
    return 'Высокий'
bank_data['income_category'] = bank_data['total_income'].apply(select_income_category)
bank_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Покупка недвижимости,Высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Покупка автомобиля,Средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Покупка недвижимости,Средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Оплата обучения,Высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Проведение мероприятий,Высокий


In [29]:
pivot_income_debt = bank_data.pivot_table(values = ['debt'], aggfunc = 'mean', index = ['income_category'], fill_value=0)
pivot_income_debt

,debt
income_category,
Высокий,0.076815
Низкий,0.061995
Средний,0.085561


In [30]:
education_df = bank_data.drop_duplicates(["education_id"])
education_dict = education_df.set_index(['education_id'])['education'].to_dict()
education_dict

{0: 'высшее',
 1: 'среднее',
 2: 'неоконченное высшее',
 3: 'начальное',
 4: 'ученая степень'}

In [31]:
dict(zip(bank_data['family_status_id'], bank_data['family_status']))

{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'Не женат / не замужем'}

In [32]:
pivot_education_debt = bank_data.pivot_table(values = ['debt'], aggfunc = 'mean', index = ['education'], fill_value=0)
pivot_education_debt

,debt
education,
высшее,0.053185
начальное,0.110320
неоконченное высшее,0.091644
среднее,0.089703
ученая степень,0.000000


In [33]:
def select_years_category(years):
    if years <= 25:
        return 'от 0 до 25'
    if years <= 50:
        return 'от 25 до 50'
    return 'от 50 и старше'
bank_data['years_category'] = bank_data['dob_years'].apply(select_years_category)
bank_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_category,years_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Покупка недвижимости,Высокий,от 25 до 50
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Покупка автомобиля,Средний,от 25 до 50
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Покупка недвижимости,Средний,от 25 до 50
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Оплата обучения,Высокий,от 25 до 50
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Проведение мероприятий,Высокий,от 50 и старше


In [34]:
pivot_years_debt = bank_data.pivot_table(values = ['debt'], aggfunc = 'mean', index = ['years_category'], fill_value=0)
pivot_years_debt

,debt
years_category,
от 0 до 25,0.104748
от 25 до 50,0.090529
от 50 и старше,0.057216


**Вывод**

Разбила датафрейм на несколько маленьких сводных таблиц, чтобы посмотреть с чем связано появление задолженности по кредиту. 

## Выявление зависимостей в данных

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [35]:
pivot_children_debt

,debt
children,
0,0.075438
1,0.091658
2,0.094542
3,0.081818


In [36]:
print('Самые ненадежные заемщики это люди с {0} детьми, процент невозвратов для них состовляет {1:.2%}'.format(pivot_children_debt.idxmax()[0],pivot_children_debt.max()[0]))
print('Самые надежные заемщики это люди с {0} детьми, процент невозвратов для них состовляет {1:.2%}'.format(pivot_children_debt.idxmin()[0],pivot_children_debt.min()[0]))


Самые ненадежные заемщики это люди с 2 детьми, процент невозвратов для них состовляет 9.45%
Самые надежные заемщики это люди с 0 детьми, процент невозвратов для них состовляет 7.54%


**Вывод**

Наличие детей не влияет на возврат кредита в срок. 
Если бы была возможность знать не только количество, но и возраст детей, то результат был бы более точным и интересным.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [37]:
pivot_family_status_debt

,debt
family_status,
Не женат / не замужем,0.097178
в разводе,0.070470
вдовец / вдова,0.066038
гражданский брак,0.093243
женат / замужем,0.075486


In [38]:
print('Самые ненадежные заемщики это люди в статусе "{0}", процент невозвратов для них состовляет {1:.2%}'.format(pivot_family_status_debt.idxmax()[0],pivot_family_status_debt.max()[0]))
print('Самые надежные заемщики это люди в статусе "{0}", процент невозвратов для них состовляет {1:.2%}'.format(pivot_family_status_debt.idxmin()[0],pivot_family_status_debt.min()[0]))


Самые ненадежные заемщики это люди в статусе "Не женат / не замужем", процент невозвратов для них состовляет 9.72%
Самые надежные заемщики это люди в статусе "вдовец / вдова", процент невозвратов для них состовляет 6.60%


In [39]:
print('Создаю словарь:')
family_dict = bank_data[['family_status_id', 'family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
display(family_dict)

print('\n\nСгруппированная таблица. Беру по id, другой столбец удалила:')
a = bank_data.groupby('family_status_id')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
display(a)

print('\n\nЗаменяю численные значения по ключу словаря:')
a.reset_index().replace({'family_status_id': family_dict.family_status.to_dict()})

Создаю словарь:


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем




Сгруппированная таблица. Беру по id, другой столбец удалила:


,count,sum,<lambda_0>
family_status_id,,,
0,12254,925,7.55%
1,4129,385,9.32%
2,954,63,6.60%
3,1192,84,7.05%
4,2799,272,9.72%




Заменяю численные значения по ключу словаря:


,family_status_id,count,sum,<lambda_0>
0,женат / замужем,12254,925,7.55%
1,гражданский брак,4129,385,9.32%
2,вдовец / вдова,954,63,6.60%
3,в разводе,1192,84,7.05%
4,Не женат / не замужем,2799,272,9.72%


**Вывод**

Согласно таблице заемщики,не состоящие в официальном браке или состоящие в глажданском браке, чаще склонны к задолженностям, чем другие группы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
pivot_income_debt

,debt
income_category,
Высокий,0.076815
Низкий,0.061995
Средний,0.085561


In [41]:
print('Самые ненадежные заемщики это люди с уровнем дохода "{0}", процент невозвратов для них состовляет {1:.2%}'.format(pivot_income_debt.idxmax()[0],pivot_income_debt.max()[0]))
print('Самые надежные заемщики это люди с уровнем дохода "{0}", процент невозвратов для них состовляет {1:.2%}'.format(pivot_income_debt.idxmin()[0],pivot_income_debt.min()[0]))


Самые ненадежные заемщики это люди с уровнем дохода "Средний", процент невозвратов для них состовляет 8.56%
Самые надежные заемщики это люди с уровнем дохода "Низкий", процент невозвратов для них состовляет 6.20%


**Вывод**

Люди с низким доходом отдают кредит в срок чаще, чем остальные группы.

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
pivot_purpose_debt

,debt
purpose_category,
Оплата обучения,0.092481
Покупка автомобиля,0.093502
Покупка недвижимости,0.073147
Проведение мероприятий,0.079186
Ремонт недвижимости,0.057851


In [43]:
print('Самые ненадежные заемщики это люди, берущие кредиты на "{0}", процент невозвратов для них состовляет {1:.2%}'.format(pivot_purpose_debt.idxmax()[0],pivot_purpose_debt.max()[0]))
print('Самые надежные заемщики это люди, берущие кредиты на "{0}", процент невозвратов для них состовляет {1:.2%}'.format(pivot_purpose_debt.idxmin()[0],pivot_purpose_debt.min()[0]))


Самые ненадежные заемщики это люди, берущие кредиты на "Покупка автомобиля", процент невозвратов для них состовляет 9.35%
Самые надежные заемщики это люди, берущие кредиты на "Ремонт недвижимости", процент невозвратов для них состовляет 5.79%


**Вывод**

Разница не очень большая, но покупка автомобиля и оплата обучения чаще приводят к задолженностям по кредиту. 

## Шаг 4. Общий вывод

In [44]:
pivot_years_debt = bank_data.pivot_table(values = ['debt'], aggfunc = 'mean', index = ['years_category', 'family_status'], fill_value=0)
pivot_years_debt.sort_values(by=['debt'], ascending=False)

debt
years_category family_status                  
от 0 до 25     вдовец / вдова         0.166667
               в разводе              0.125000
               Не женат / не замужем  0.118609
от 25 до 50    гражданский брак       0.103896
               Не женат / не замужем  0.103347
от 0 до 25     гражданский брак       0.095679
               женат / замужем        0.094538
от 25 до 50    женат / замужем        0.085064
               в разводе              0.076087
               вдовец / вдова         0.074257
от 50 и старше гражданский брак       0.066667
               вдовец / вдова         0.063003
               Не женат / не замужем  0.062603
               в разводе              0.056604
               женат / замужем        0.052448

Я решила сопоставить возраст и семейное положение заемщиков и вот что получилось: люди до 50 лет, не состоящие в официальном браке - наиболее неподходящая для одобрения кредита группа граждан. Наличие детей не влияет на погашение кредита в срок, в отличие от наличия законного супруга(супруги).